In [48]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn

from torch.utils.data import Dataset


class XORData(Dataset):
    def __init__(self, length, batch_size):
        super().__init__()

        self.length = length
        self.batch_size = batch_size

        self.data = torch.Tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
        self.targets = torch.Tensor([[0.0], [1.0], [1.0], [1.0]])

    def __getitem__(self, index):
        inputs = torch.randint(
            low=0,
            high=2,
            size=(self.batch_size, 2),
        )
        targets = torch.bitwise_xor(inputs[:, 0], inputs[:, 1])

        return inputs, targets.unsqueeze_(dim=1)

    def __len__(self):
        return self.length // self.batch_size


class XORModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_layer = nn.Linear(2, 3, bias=False)
        self.activation1 = nn.Tanh()
        self.output_layer = nn.Linear(3, 1, bias=False)

    def forward(self, x, print_bool=False):

        if print_bool==True:
            x = self.hidden_layer(x)
            x = self.activation1(x)
            print(x)
            x = self.output_layer(x)
        else:
            x = self.hidden_layer(x)
            x = self.activation1(x)
            x = self.output_layer(x)
        return x

data = XORData(10000, 10)
data_loader = DataLoader(data, batch_size=1)


In [49]:
model = XORModel()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.L1Loss(reduction='mean', reduce=True)

for epoch in range(12):
    train_loss = 0
    for batch in data_loader:
        input, target = batch

        optimizer.zero_grad()
        output = model(input.float())
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print('epoch: {} | loss: {:.8f}'.format(
        epoch + 1,
        train_loss / len(data_loader)
    ))

/home/heinrich/projects/language_model_pytorch/.venv/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


epoch: 1 | loss: 0.10023623
epoch: 2 | loss: 0.00374301
epoch: 3 | loss: 0.00306273
epoch: 4 | loss: 0.00311355
epoch: 5 | loss: 0.00326434
epoch: 6 | loss: 0.00347490
epoch: 7 | loss: 0.00327510
epoch: 8 | loss: 0.00321626
epoch: 9 | loss: 0.00368590
epoch: 10 | loss: 0.00328628
epoch: 11 | loss: 0.00299241
epoch: 12 | loss: 0.00350467


In [50]:
for name, param in model.named_parameters():
    print(name)
    print(param)

hidden_layer.weight
Parameter containing:
tensor([[-2.3912,  0.7140],
        [ 0.3000, -1.6301],
        [-3.0067, -2.7479]], requires_grad=True)
output_layer.weight
Parameter containing:
tensor([[ 0.6823,  0.8896, -1.4194]], requires_grad=True)


In [51]:
input = torch.Tensor([[0.0, 0.0],[0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
target = torch.Tensor([0.0, 1.0, 1.0, 0.0])
output = model(torch.Tensor([1.0, 1.0]), print_bool=True)


tensor([-0.9325, -0.8693, -1.0000], grad_fn=<TanhBackward>)


In [ ]:
criterion = nn.MSELoss()
criterion(output, target.unsqueeze_(dim=1))
criterion(torch.Tensor([0.75]), torch.Tensor([1.0]))